# Epileptic Seizure Classification with Bi-LSTM & Attention-Layer
This notebook contains the classification of time series EEG data for the detection of epileptic seizures based on the preprocessed CHB-MIT Scalp EEG Database.
The codes is structured as followed:
1. [Imports](#1-imports)
2. [Load Preprocessed Dataset](#2-load-preprocessed-dataset)
3. [Split Dataset](#3-split-dataset)
4. [Normalize Dataset](#4-normalize-dataset)
5. [Bi-LSTM Model](#5-autoencoder) <br>
5.1 [Define Neural Network-Model](#52-define-autoencoder-model) <br>
5.2 [Compile Neural Network-Model]() <br>
5.3 [Train Neural Network]() <br>
6. [Validate Results](#6-validate-results)
7. [Explain Model with SHAP]()
8. [Conclusions](#8-conclusion)

## 1. Imports
Import requiered libraries. <br>
External packages can be installed via the `pip install` command.

In [ ]:
import numpy as np

# Pre-Processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Neural Network
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input, Attention
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import L1L2

import plotly.graph_objects as go
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from imblearn.metrics import geometric_mean_score

# from tqdm.keras import TqdmCallback

## 2. Load Preprocessed Dataset
In order to load the preprocessed dataset, that was created with the notebook `00_Preprocessing.ipynb`, is loaded and the numpy Arrays for the features and labels are extracted. <br>
To enshure a functional distribution of the classes in the dataset, the classes with the respective amounts are plotted.

In [ ]:
dataset = np.load('../00_Data/Processed-Data/classification_dataset_max.npz')
X = dataset["features"]
y = dataset["labels"]

## 3. Split Dataset
In order to validate and test the trained classifier, the dataset must be split into a `train`, `test`, and `validation` subset. <br>
To preserve an equal distribution within each split, the `stratify`-option is enabled.

In [ ]:
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.4, shuffle=True, stratify=np.ravel(y), random_state=34)
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, test_size=0.5, shuffle=True, stratify=np.ravel(y_rest), random_state=34)

## 4. Normalize Dataset
When working with neural networks, it is imperative to normalize the data bevore training and testing. This enshures a faster training, avoids numerical instablities and provides a better generalization of the neural network. However with EEG-data, there are additional requirements due to the different characteristics and value-ranges of the individual channels. Therefore, the normalization is done channel by channel based on the training-subset and applied on the test- and validation-split.

In [ ]:
def normalize_features(X_train:np.ndarray, X_test:np.ndarray, X_val:np.ndarray, use_standard_scaler:bool=False) -> tuple:
    if(use_standard_scaler):
        scaler = StandardScaler()
    else:
        scaler = MinMaxScaler()
    X_train_norm = np.zeros(shape=(X_train.shape), dtype='float32')
    X_test_norm = np.zeros(shape=(X_test.shape), dtype='float32')
    X_val_norm = np.zeros(shape=(X_val.shape), dtype='float32')
    for feature_col in range(X_train.shape[2]):
        X_train_norm[:][:][feature_col] = scaler.fit_transform(X_train[:][:][feature_col])
        X_test_norm[:][:][feature_col] = scaler.transform(X_test[:][:][feature_col])
        X_val_norm[:][:][feature_col] = scaler.transform(X_val[:][:][feature_col])
    return X_train_norm, X_test_norm, X_val_norm

In [ ]:
X_train_norm, X_test_norm, X_val_norm = normalize_features(X_train, X_test, X_val, True)

## 5. Bi-LSTM Model
This section contains the definition, compilation & training of the neural network.

### 5.1 Define Neural Network-Model
The neural network consists out of Bi-LSTM layers followed by an attention layer.

In [ ]:
def build_and_compile_model(train_shape:tuple, initial_lr:float=0.0001):
    inputs = Input(shape=(train_shape[1], train_shape[2]))
    bilstm1 = Bidirectional(LSTM(64, return_sequences = True, kernel_regularizer=L1L2(0, 0.001)))(inputs)
    do1 = Dropout(0.5)(bilstm1)
    bilstm2 = Bidirectional(LSTM(32, return_sequences = True, kernel_regularizer=L1L2(0, 0.001)))(do1)
    at = Attention(32)([bilstm2, bilstm2])
    d1 = Dense(16, kernel_regularizer=L1L2(0, 0.001))(at)
    do3 = Dropout(0.5)(d1)
    d2 = Dense(8, kernel_regularizer=L1L2(0, 0.0001))(do3)
    outputs = Dense(1, activation='sigmoid')(d2)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    opt = tf.keras.optimizers.legacy.Adam(learning_rate=initial_lr)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])
    return model

### 5.2 Compile Neural Network-Model

In [ ]:
model = build_and_compile_model(X_train.shape, 0.0001)

earlystopper = EarlyStopping(patience=25, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0000001, verbose=1, cooldown=10)

model.summary()

### 5.3 Fit Neural Network-Model

In [ ]:
history = model.fit(
    X_train, 
    y_train, 
    epochs=50, 
    batch_size=50,
    validation_data=(X_val, y_val),
    verbose=1, 
    callbacks=[earlystopper, reduce_lr] #, TqdmCallback(verbose=2)],
)

In [ ]:
fig = go.Figure(
    data = [
        go.Scatter(y=history.history['loss'], name="train"),
        go.Scatter(y=history.history['val_loss'], name="val"),
    ],
    layout = {"yaxis": {"title": "Loss [MSE]"}, "xaxis": {"title": "Epoch"}, "title": "Model Loss over Epochs"}
)

fig.show()

## 6. Validate Results

In [ ]:
y_test_predictions = model.predict(X_test)
y_test_predictions = (y_test_predictions >= 0.5).astype(int)
f1score = f1_score(y_test, y_test_predictions)
gm = geometric_mean_score(y_test, y_test_predictions, average="binary")
auc = roc_auc_score(y_test, y_test_predictions, average="weighted")
precision = precision_score(y_test, y_test_predictions)
recall = recall_score(y_test, y_test_predictions)

In [ ]:
a = []
for i in range(len(y_test_predictions)):
    a.append(y_test_predictions[i][0][0])

In [ ]:
y_test_predictions2 = (np.array(a) >= 0.5).astype(int)
f1score = f1_score(y_test, y_test_predictions2)
gm = geometric_mean_score(y_test, y_test_predictions2, average="binary")
auc = roc_auc_score(y_test, y_test_predictions2, average="weighted")
precision = precision_score(y_test, y_test_predictions2)
recall = recall_score(y_test, y_test_predictions2)

In [ ]:
print(f1score, gm, auc, precision, recall)

In [ ]:
# Max      0.8529284164859002 0.8558157521673803 0.8558649977293616 0.8592657342657343 0.8466838931955211
# Majority 0.7218001168907071 0.7578027296919185 0.774371772170577 0.8734087694483734 0.6150398406374502
# Max+Atte 0.9416648459689753 0.9429646733706349 0.9430840468336327 0.9556541019955654 0.9280792420327304